## Setting Up:

In [ ]:
import sys
import os

import logging
import warnings
from datetime import datetime
from collections import defaultdict, Counter
from functools import partial

import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from tqdm.notebook import tqdm
from cmcrameri import cm
from skorch.helper import SliceDataset
from skorch.callbacks import EarlyStopping, LRScheduler, Checkpoint

import massbalancemachine as mbm

# Add root of repo to import MBM
sys.path.append(os.path.join(os.getcwd(), '../../'))

# Local modules
from scripts.helpers import *
from scripts.glamos_preprocess import *
from scripts.plots import *
from scripts.config_CH import *
from scripts.nn_helpers import *
from scripts.xgb_helpers import *
from scripts.geodata import *
from scripts.NN_networks import *
from scripts.geodata_plots import *

warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

cfg = mbm.SwitzerlandConfig()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.data import WeightedRandomSampler, SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

seed_all(cfg.seed)
print("Using seed:", cfg.seed)

if torch.cuda.is_available():
    print("CUDA is available")
    free_up_cuda()
else:
    print("CUDA is NOT available")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Plot styles:
path_style_sheet = 'scripts/example.mplstyle'
plt.style.use(path_style_sheet)
colors = get_cmap_hex(cm.batlow, 10)
color_dark_blue = colors[0]
color_pink = '#c51b7d'

# RGI Ids:
# Read rgi ids:
rgi_df = pd.read_csv(cfg.dataPath + path_glacier_ids, sep=',')
rgi_df.rename(columns=lambda x: x.strip(), inplace=True)
rgi_df.sort_values(by='short_name', inplace=True)
rgi_df.set_index('short_name', inplace=True)
vois_climate = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
]

vois_topographical = [
    "aspect_sgi", "slope_sgi", "hugonnet_dhdt", "consensus_ice_thickness",
    "millan_v", "svf"
]

## Input data:

In [ ]:
data_glamos = getStakesData(cfg)

POINT_ID_to_drop = [
    'schwarzberg_12', 'schwarzberg_13', 'schwarzberg_6', 'schwarzberg_5',
    'schwarzberg_12', 'plattalva_7', 'plattalva_11', 'plattalva_10', 'plattalva_1'
]

# remove points with too much missing data
data_glamos = data_glamos[~data_glamos['POINT_ID'].isin(POINT_ID_to_drop)]

months_head_pad, months_tail_pad = mbm.data_processing.utils._compute_head_tail_pads_from_df(
    data_glamos)

# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')

# Transform data to monthly format (run or load data):
paths = {
    'csv_path': cfg.dataPath + path_PMB_GLAMOS_csv,
    'era5_climate_data':
    cfg.dataPath + path_ERA5_raw + 'era5_monthly_averaged_data.nc',
    'geopotential_data':
    cfg.dataPath + path_ERA5_raw + 'era5_geopotential_pressure.nc',
    'radiation_save_path': cfg.dataPath + path_pcsr + 'zarr/'
}
RUN = True
data_monthly = process_or_load_data(
    run_flag=RUN,
    data_glamos=data_glamos,
    paths=paths,
    cfg=cfg,
    vois_climate=vois_climate,
    vois_topographical=vois_topographical,
    output_file='CH_wgms_dataset_monthly_LSTM_svf.csv')

# Create DataLoader
dataloader_gl = mbm.dataloader.DataLoader(cfg,
                                          data=data_monthly,
                                          random_seed=cfg.seed,
                                          meta_data_columns=cfg.metaData)

In [ ]:
# Ensure all test glaciers exist in the dataset
existing_glaciers = set(data_monthly.GLACIER.unique())
missing_glaciers = [g for g in TEST_GLACIERS if g not in existing_glaciers]

# Define training glaciers correctly
train_glaciers = [i for i in existing_glaciers if i not in TEST_GLACIERS]

data_test = data_monthly[data_monthly.GLACIER.isin(TEST_GLACIERS)]
data_train = data_monthly[data_monthly.GLACIER.isin(train_glaciers)]
splits, test_set, train_set = get_CV_splits(dataloader_gl,
                                            test_split_on='GLACIER',
                                            test_splits=TEST_GLACIERS,
                                            random_state=cfg.seed)
# Validation and train split:
data_train = train_set['df_X']
data_train['y'] = train_set['y']
data_test = test_set['df_X']
data_test['y'] = test_set['y']

## LSTM:

In [ ]:
MONTHLY_COLS = [
    't2m',
    'tp',
    'slhf',
    'sshf',
    'ssrd',
    'fal',
    'str',
    'ELEVATION_DIFFERENCE',
]
STATIC_COLS = [
    'aspect_sgi', 'slope_sgi', 'hugonnet_dhdt', 'consensus_ice_thickness',
    'millan_v', 'svf'
]
feature_columns = MONTHLY_COLS + STATIC_COLS
cfg.setFeatures(feature_columns)

### Build LSTM dataloaders:

In [ ]:
seed_all(cfg.seed)

df_train = data_train.copy()
df_train['PERIOD'] = df_train['PERIOD'].str.strip().str.lower()

df_test = data_test.copy()
df_test['PERIOD'] = df_test['PERIOD'].str.strip().str.lower()

# --- build train dataset from dataframe ---
ds_train = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_train,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True)

ds_test = mbm.data_processing.MBSequenceDataset.from_dataframe(
    df_test,
    MONTHLY_COLS,
    STATIC_COLS,
    months_tail_pad=months_tail_pad,
    months_head_pad=months_head_pad,
    expect_target=True)

train_idx, val_idx = mbm.data_processing.MBSequenceDataset.split_indices(
    len(ds_train), val_ratio=0.2, seed=cfg.seed)

### Define & train model:

In [ ]:
custom_params = {
    'Fm': 8,
    'Fs': 6,
    'hidden_size': 128,
    'num_layers': 2,
    'bidirectional': False,
    'dropout': 0.1,
    'static_layers': 2,
    'static_hidden': [128, 64],
    'static_dropout': 0.1,
    'lr': 0.001,
    'weight_decay': 0.0,
    'loss_name': 'neutral',
    'two_heads': True,
    'head_dropout': 0.0,
    'loss_spec': None
}

# --- loaders (fit scalers on TRAIN, apply to whole ds_train) ---
ds_train_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_train)

ds_test_copy = mbm.data_processing.MBSequenceDataset._clone_untransformed_dataset(
    ds_test)

train_dl, val_dl = ds_train_copy.make_loaders(
    train_idx=train_idx,
    val_idx=val_idx,
    batch_size_train=64,
    batch_size_val=128,
    seed=cfg.seed,
    fit_and_transform=
    True,  # fit scalers on TRAIN and transform Xm/Xs/y in-place
    shuffle_train=True,
    use_weighted_sampler=True  # use weighted sampler for training
)

# --- test loader (copies TRAIN scalers into ds_test and transforms it) ---
test_dl = mbm.data_processing.MBSequenceDataset.make_test_loader(
    ds_test_copy, ds_train_copy, batch_size=128, seed=cfg.seed)

# --- build model, resolve loss, train, reload best ---
model = mbm.models.LSTM_MB.build_model_from_params(cfg, custom_params, device)
loss_fn = mbm.models.LSTM_MB.resolve_loss_fn(custom_params)

# Evaluate on test
model_filename = f"models/lstm_model_2025-10-16_two_heads_svf_OOS.pt"
state = torch.load(model_filename, map_location=device)
model.load_state_dict(state)
test_metrics, test_df_preds = model.evaluate_with_preds(
    device, test_dl, ds_test_copy)
test_rmse_a, test_rmse_w = test_metrics['RMSE_annual'], test_metrics[
    'RMSE_winter']

print('Test RMSE annual: {:.3f} | winter: {:.3f}'.format(
    test_rmse_a, test_rmse_w))

## Validation OOS:
Out of sample on test set

In [ ]:
colors = get_cmap_hex(cm.batlow, 10)
color_annual = "#c51b7d"
color_winter = colors[0]

In [ ]:
scores_annual, scores_winter = compute_seasonal_scores(test_df_preds,
                                                       target_col='target',
                                                       pred_col='pred')

print("Annual scores:", scores_annual)
print("Winter scores:", scores_winter)

fig = plot_predictions_summary(
    grouped_ids=test_df_preds,
    scores_annual=scores_annual,
    scores_winter=scores_winter,
    ax_xlim=(-8, 6),
    ax_ylim=(-8, 6),
    color_annual=color_annual,
    color_winter=color_winter,
)

In [ ]:
# Areas (with clariden alias fix)
gl_area = get_gl_area(cfg)
gl_area["clariden"] = gl_area["claridenL"]

In [ ]:
gl_per_el = data_glamos[data_glamos.PERIOD == 'annual'].groupby(
    ['GLACIER'])['POINT_ELEVATION'].mean()
gl_per_el = gl_per_el.sort_values(ascending=False)

test_gl_per_el = gl_per_el[TEST_GLACIERS].sort_values().index

fig, axs = plt.subplots(3, 3, figsize=(25, 18), sharex=True)

gl_per_el = data_glamos[data_glamos.PERIOD == 'annual'].groupby(
    ['GLACIER'])['POINT_ELEVATION'].mean()
gl_per_el = gl_per_el.sort_values(ascending=False)
test_df_preds['gl_elv'] = test_df_preds['GLACIER'].map(gl_per_el)

subplot_labels = [
    '(a)', '(b)', '(c)', '(d)', '(e)', '(f)', '(g)', '(h)', '(i)'
]

axs = PlotIndividualGlacierPredVsTruth(test_df_preds,
                                       axs=axs,
                                       subplot_labels=subplot_labels,
                                       color_annual=color_annual,
                                       color_winter=color_winter,
                                       custom_order=test_gl_per_el, 
                                       gl_area = gl_area)

axs[3].set_ylabel("Modelled PMB [m w.e.]", fontsize=20)

fig.supxlabel('Observed PMB [m w.e.]', fontsize=20, y=0.06)
# two distinct handles
legend_scatter_annual = Line2D([0], [0],
                               marker='o',
                               linestyle='None',
                               linewidth=0,
                               markersize=10,
                               markerfacecolor=color_annual,
                               markeredgecolor='k',
                               markeredgewidth=0.8,
                               label='Annual')

legend_scatter_winter = Line2D([0], [0],
                               marker='o',
                               linestyle='None',
                               linewidth=0,
                               markersize=10,
                               markerfacecolor=color_winter,
                               markeredgecolor='k',
                               markeredgewidth=0.8,
                               label='Winter')

# if you already have other handles (e.g., bands/means), append these:
# handles = existing_handles + [legend_scatter_annual, legend_scatter_winter]
handles = [legend_scatter_annual, legend_scatter_winter]

# You can let matplotlib use the labels from the handles; no need to pass `labels=...`
fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=20)

plt.subplots_adjust(hspace=0.25, wspace=0.25)
plt.show()

## Intermediate validation OOS and IS:

In [ ]:
# Geodetic MB + per-glacier periods
geodetic_mb = get_geodetic_MB(cfg)
periods_per_glacier, geoMB_per_glacier = build_periods_per_glacier(geodetic_mb)

In [ ]:
PATH_PREDICTIONS_LSTM_OOS = os.path.join(cfg.dataPath, "GLAMOS",
                                         "distributed_MB_grids",
                                         "MBM/glamos_dems_LSTM_svf_OOS")

PATH_PREDICTIONS_LSTM_IS = os.path.join(cfg.dataPath, "GLAMOS",
                                        "distributed_MB_grids",
                                        "MBM/glamos_dems_LSTM_svf_IS")

In [ ]:
# Available glaciers (those with LSTM predictions)
glaciers_in_glamos = set(os.listdir(PATH_PREDICTIONS_LSTM_OOS))

# Areas (with clariden alias fix)
gl_area = get_gl_area(cfg)
gl_area["clariden"] = gl_area["claridenL"]

# Glaciers present in both geodetic periods and predictions, sorted by area (asc)
glacier_list = sorted(
    (g for g in periods_per_glacier.keys() if g in glaciers_in_glamos),
    key=lambda g: gl_area.get(g, 0))
print("Number of glaciers:", len(glacier_list))
print("Glaciers:", glacier_list)

In [ ]:
# Run comparison
ds_lstm_OS = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=os.path.join(cfg.dataPath, path_SMB_GLAMOS_csv),
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=TEST_GLACIERS,
    path_predictions=PATH_PREDICTIONS_LSTM_OOS,
    cfg=cfg,
)

# Drop rows where any required columns are NaN
ds_lstm_OS = ds_lstm_OS.dropna(subset=['Geodetic MB', 'MBM MB'])
ds_lstm_OS = ds_lstm_OS.sort_values(by='Area')
ds_lstm_OS['GLACIER'] = ds_lstm_OS['GLACIER'].apply(lambda x: x.capitalize())

In [ ]:
# Run comparison
ds_lstm_IS = process_geodetic_mass_balance_comparison(
    glacier_list=glacier_list,
    path_SMB_GLAMOS_csv=os.path.join(cfg.dataPath, path_SMB_GLAMOS_csv),
    periods_per_glacier=periods_per_glacier,
    geoMB_per_glacier=geoMB_per_glacier,
    gl_area=gl_area,
    test_glaciers=TEST_GLACIERS,
    path_predictions=PATH_PREDICTIONS_LSTM_IS,
    cfg=cfg,
)

# Drop rows where any required columns are NaN
ds_lstm_IS = ds_lstm_IS.dropna(subset=['Geodetic MB', 'MBM MB'])
ds_lstm_IS = ds_lstm_IS.sort_values(by='Area')
ds_lstm_IS['GLACIER'] = ds_lstm_IS['GLACIER'].apply(lambda x: x.capitalize())

### Mass balance gradients:

In [ ]:
# areas_per_gl = ds_lstm_OS.groupby(
#     'GLACIER').Area.mean().reset_index().set_index('GLACIER')

# Stake data
# Load stake data ONCE instead of for every glacier
stake_file = os.path.join(cfg.dataPath, path_PMB_GLAMOS_csv,
                          "CH_wgms_dataset_all.csv")
df_stakes = pd.read_csv(stake_file)

#### Gradients all:

In [ ]:
nrows = 4
ncols = 5
cm = 1 / 2.54
fontsize = 7

# bin0 = ['Schwarzbach', 'Sexrouge', 'Murtel']
# bin1 = ['Basodino', 'Adler', 'Hohlaub', 'Silvretta', 'Gries', 'Clariden']
# bin2 = ['Gietro', 'Schwarzberg', 'Allalin']
# bin3 = ['Findelen', 'Rhone', 'Aletsch']
# glaciers = bin0 + bin1 + bin2 + bin3

# Create a figure with the specified number of subplots
fig, axs = plt.subplots(nrows=nrows,
                        ncols=ncols,
                        figsize=(25 * cm, 15 * cm),
                        dpi=300)
axs = axs.flatten()
gl_list = [
    'schwarzbach', 'murtel', 'plattalva', 'basodino', 'limmern', 'adler',
    'hohlaub', 'albigna', 'tsanfleuron', 'silvretta', 'gries', 'clariden',
    'gietro', 'schwarzberg', 'forno', 'allalin', 'otemma', 'findelen', 'rhone',
    'aletsch'
]
for i, gl in enumerate(gl_list):
    # Annual
    df_lstm_a, df_glamos_a, df_all_a = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="annual")

    years = df_all_a.YEAR.unique()

    # Winter
    df_lstm_w, df_glamos_w, df_all_w = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="winter")

    # if dataframe not None
    if df_all_a.empty:
        print(f"No data for glacier: {gl}")
        continue

    ax = plot_mb_by_elevation_periods_combined(df_all_a,
                                               df_all_w,
                                               df_stakes,
                                               gl.lower(),
                                               ax=axs[i])

    # area = areas_per_gl.loc[gl].Area
    area = gl_area.get(gl.lower(), np.nan)
    if area < 0.1:
        area = np.round(area, 3)
    else:
        area = np.round(area, 1)
    if gl.lower() in TEST_GLACIERS:
        axs[i].set_title(f'*{gl} ({area} km2, {years.min()}-{years.max()})',
                         fontsize=fontsize,
                         pad=2)
    else:
        axs[i].set_title(f'{gl} ({area} km2, {years.min()}-{years.max()})',
                         fontsize=fontsize,
                         pad=2)

    axs[i].grid(alpha=0.2)
    axs[i].tick_params(labelsize=6.5, pad=2)
    axs[i].set_ylabel('')
    axs[i].set_xlabel('')
    # remove legend
    axs[i].legend().remove()

axs[5].set_ylabel('Elevation (m a.s.l.)', fontsize=fontsize)

# Custom handles (bands, means, and stakes)
handles = [
    # LSTM
    Patch(facecolor=color_annual, alpha=0.25, label="LSTM band (annual)"),
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='-',
           label="LSTM mean (annual)"),
    Patch(facecolor=color_winter, alpha=0.25, label="LSTM band (winter)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='-',
           label="LSTM mean (winter)"),

    # GLAMOS (mean only)
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (winter)"),

    # Stakes means
    Line2D([0], [0],
           marker='o',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_annual,
           markeredgewidth=1.2,
           label="Stakes mean (annual)"),
    Line2D([0], [0],
           marker='s',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_winter,
           markeredgewidth=1.2,
           label="Stakes mean (winter)"),
]

fig.supxlabel('Mass balance (m w.e.)', fontsize=fontsize, y=0.06)

fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=4,
           fontsize=7)

# Adjust the layout
plt.subplots_adjust(hspace=0.25, wspace=0.25)
plt.show()

#### Gradients comparison:

In [ ]:
gl_list = [
    'Hohlaub',
    'Tsanfleuron',
    'Schwarzberg',
    'Forno',
]

nrows = 1  # 0: OOS, 1: IS
ncols = len(gl_list)
cm = 1 / 2.54
fontsize = 7

row_labels = ['Out-of-sample', 'In-sample']

fig, axs = plt.subplots(nrows=nrows,
                        ncols=ncols,
                        figsize=(25 * cm, 12 * cm),
                        dpi=300)

for c, gl in enumerate(gl_list):  # columns = glaciers
    # Annual
    df_lstm_a_oos, df_glamos_a_oos, df_all_a_oos = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_OOS, cfg, period="annual")
    # Winter
    df_lstm_w_oos, df_glamos_w_oos, df_all_w_oos = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_OOS, cfg, period="winter")

    # Annual
    df_lstm_a_is, df_glamos_a_is, df_all_a_is = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="annual")
    # Winter
    df_lstm_w_is, df_glamos_w_is, df_all_w_is = build_all_years_df(
        gl.lower(), PATH_PREDICTIONS_LSTM_IS, cfg, period="winter")

    # get unique years
    years = df_all_w_oos.YEAR.unique()

    ax = axs[c]

    # OOS: bands + mean + GLAMOS + stakes
    ax = plot_lstm_by_elevation_periods(df_all_a_oos,
                                        df_all_w_oos,
                                        ax=ax,
                                        mean_linestyle='--',
                                        label_prefix='LSTM OOS',
                                        show_band=False,
                                        color_annual=color_annual,
                                        color_winter=color_winter)

    # IS: LSTM mean-only overlay (no band), dashed line to distinguish
    ax = plot_lstm_by_elevation_periods(df_all_a_is,
                                        df_all_w_is,
                                        ax=ax,
                                        mean_linestyle='-',
                                        label_prefix='LSTM IS',
                                        show_band=True,
                                        color_annual=color_annual,
                                        color_winter=color_winter)

    ax = plot_glamos_by_elevation_periods(df_all_a_oos,
                                          df_all_w_oos,
                                          ax=ax,
                                          show_band=False,
                                          label_prefix="GLAMOS",
                                          mean_linestyle=":",
                                          color_annual=color_annual,
                                          color_winter=color_winter)

    # add stakes:
    ax = plot_stakes_by_elevation_periods(df_stakes,
                                          gl.lower(),
                                          valid_bins=None,
                                          ax=ax,
                                          color_annual=color_annual,
                                          color_winter=color_winter,
                                          marker_size=14)

    ax.set_ylabel('')
    ax.set_xlabel('')

    area = gl_area.get(gl.lower(), np.nan)
    area = np.round(area, 3) if area < 0.1 else np.round(area, 1)

    ax.set_title(f'{gl} ({area} km2, {years.min()}-{years.max()})',
                 fontsize=fontsize,
                 pad=2)

    # Row label on the left margin (first column only)
    if c == 0:
        ax.set_ylabel(f'Elevation (m a.s.l.)', fontsize=fontsize)

    ax.grid(alpha=0.2)
    ax.tick_params(labelsize=6.5, pad=2)
    ax.set_xlabel('')  # we use a supxlabel below

    # remove per-axes legend
    leg = ax.legend()
    if leg is not None:
        leg.remove()

# Global x-label
fig.supxlabel('Mass balance (m w.e.)', fontsize=fontsize, y=0.06)

# Custom handles (bands, means, and stakes)
handles = [
    # LSTM
    Patch(facecolor=color_annual,
          alpha=0.25,
          label="LSTM in-sample band (annual)"),
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='-',
           label="LSTM in-sample mean (annual)"),
    Patch(facecolor=color_winter,
          alpha=0.25,
          label="LSTM in-sample band (winter)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='-',
           label="LSTM in-sample mean (winter)"),
    # LSTM OOS (mean only)
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle='--',
           label="LSTM out-of-sample mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle='--',
           label="LSTM out-of-sample mean (winter)"),
    # LSTM IS (mean only)
    Line2D([0], [0],
           color=color_annual,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (annual)"),
    Line2D([0], [0],
           color=color_winter,
           lw=1.2,
           linestyle=':',
           label="GLAMOS mean (winter)"),

    # Stakes means
    Line2D([0], [0],
           marker='o',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_annual,
           markeredgewidth=1.2,
           label="Stakes mean (annual)"),
    Line2D([0], [0],
           marker='s',
           linestyle='None',
           linewidth=0,
           markersize=6,
           markerfacecolor='none',
           markeredgecolor=color_winter,
           markeredgewidth=1.2,
           label="Stakes mean (winter)"),
]

fig.legend(handles=handles,
           loc='upper center',
           bbox_to_anchor=(0.5, 0.05),
           ncol=5,
           fontsize=7)

plt.subplots_adjust(hspace=0.25, wspace=0.25)
plt.show()

In [ ]:
test_gl_area = {}
for x in TEST_GLACIERS:
    test_gl_area[x] = gl_area[x]
test_gl_area = dict(
    sorted(test_gl_area.items(), key=lambda item: item[1], reverse=True))
test_gl_area

### Maps:

In [ ]:
# df_nn = process_geodetic_mass_balance_comparison(
#     glacier_list=glacier_list,
#     path_SMB_GLAMOS_csv=os.path.join(cfg.dataPath, path_SMB_GLAMOS_csv),
#     periods_per_glacier=periods_per_glacier,
#     geoMB_per_glacier=geoMB_per_glacier,
#     gl_area=gl_area,
#     test_glaciers=TEST_GLACIERS,
#     path_predictions=PATH_PREDICTIONS_NN,
#     cfg=cfg,
# )

# Example usage
# # bias_gl = df[df.GLACIER == GLACIER_NAME.capitalize()].bias_gl.unique()[0]
# df_lstm_two_heads_gl = df_lstm_two_heads[df_lstm_two_heads.GLACIER ==
#                                          GLACIER_NAME.capitalize()]
# df_nn_gl = df_nn[df_nn.GLACIER == GLACIER_NAME]

# fig, axs = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)

# plot_scatter_comparison(axs[0],
#                         df_lstm_two_heads_gl,
#                         GLACIER_NAME,
#                         color_mbm=color_annual,
#                         color_glamos=color_winter,
#                         title_suffix="(LSTM two heads)")
# plot_scatter_comparison(axs[1],
#                         df_nn_gl,
#                         GLACIER_NAME,
#                         color_mbm=color_annual,
#                         color_glamos=color_winter,
#                         title_suffix="(MLP)")

# plt.tight_layout()
# plt.show()

In [ ]:
GLACIER_NAME = 'rhone'

# Load GLAMOS data
GLAMOS_glwmb = get_GLAMOS_glwmb(GLACIER_NAME, cfg)

MBM_glwmb_nn = mbm_glwd_pred(PATH_PREDICTIONS_NN, GLACIER_NAME)
MBM_glwmb_nn.rename(columns={"MBM Balance": "MBM Balance MLP"}, inplace=True)

MBM_glwmb_lstm = mbm_glwd_pred(PATH_PREDICTIONS_LSTM_IS, GLACIER_NAME)
MBM_glwmb_lstm.rename(columns={"MBM Balance": "MBM Balance LSTM"},
                      inplace=True)

# Merge with GLAMOS data
MBM_glwmb_nn = MBM_glwmb_nn.join(GLAMOS_glwmb)
MBM_glwmb_nn = MBM_glwmb_nn.dropna()

MBM_glwmb = MBM_glwmb_nn.join(MBM_glwmb_lstm)

# Plot the data
fig, axs = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
MBM_glwmb.plot(ax=axs[0],
               y=['MBM Balance LSTM', 'GLAMOS Balance'],
               marker="o",
               color=[color_annual, color_winter])
MBM_glwmb.plot(ax=axs[1],
               y=['MBM Balance MLP', 'GLAMOS Balance'],
               marker="o",
               color=[color_annual, color_winter])

for ax in axs:
    ax.set_title(f"{GLACIER_NAME.capitalize()} Glacier", fontsize=24)
    ax.set_ylabel("Mass Balance [m w.e.]", fontsize=18)
    ax.set_xlabel("Year", fontsize=18)
    ax.grid(True, linestyle="--", linewidth=0.5)
    ax.legend(fontsize=14)

axs[0].set_title(f"{GLACIER_NAME.capitalize()} Glacier (LSTM)", fontsize=16)
axs[1].set_title(f"{GLACIER_NAME.capitalize()} Glacier (MLP)", fontsize=16)

plt.tight_layout()
plt.show()

In [ ]:
for year in MBM_glwmb_nn.index:
    plot_mass_balance_comparison_annual(
        glacier_name=GLACIER_NAME,
        year=year,
        cfg=cfg,
        df_stakes=df_stakes,
        path_distributed_mb=path_distributed_MB_glamos,
        path_pred_lstm=PATH_PREDICTIONS_LSTM_two_heads,
        path_pred_nn=PATH_PREDICTIONS_NN,
        period='annual')


#### Two glaciers, two years:

In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable


def plot_2glaciers_2years_glamos_vs_lstm(
        glacier_names,  # e.g. ("aletsch", "silvretta")
        years,  # e.g. (2016, 2022)
        cfg,
        df_stakes=None,
        path_distributed_mb=None,  # GLAMOS grids base
        path_pred_lstm=None,  # LSTM zarrs base
        period="annual",
        apply_smoothing_fn=None,  # optional callable(ds)->ds
):
    """
    Layout (2 rows × 4 panels) with one colorbar per glacier–year pair (outside maps):
      Row 1: G1-GLAMOS(y1), G1-LSTM(y1), |cbar|, G1-GLAMOS(y2), G1-LSTM(y2), |cbar|
      Row 2: G2-GLAMOS(y1), G2-LSTM(y1), |cbar|, G2-GLAMOS(y2), G2-LSTM(y2), |cbar|

    Each row shares the same y-axis; only the leftmost panel in each row shows y labels/ticks.
    Annotations (lower-left of each panel): RMSE (vs stakes), mean MB, variance.
    """

    assert len(glacier_names) == 2, "glacier_names must have length 2"
    assert len(years) == 2, "years must have length 2"
    assert path_distributed_mb and path_pred_lstm

    # ---------- helpers ----------
    def pick_file_glamos(glacier, year, period="annual"):
        suffix = "ann" if period == "annual" else "win"
        base = os.path.join(cfg.dataPath, path_distributed_mb, "GLAMOS",
                            glacier)
        cand_lv95 = os.path.join(base, f"{year}_{suffix}_fix_lv95.grid")
        cand_lv03 = os.path.join(base, f"{year}_{suffix}_fix_lv03.grid")
        if os.path.exists(cand_lv95): return cand_lv95, "lv95"
        if os.path.exists(cand_lv03): return cand_lv03, "lv03"
        return None, None

    def load_glamos_wgs84(glacier, year):
        path, cs = pick_file_glamos(glacier, year, period)
        if path is None: return None
        meta, arr = load_grid_file(path)
        da = convert_to_xarray_geodata(arr, meta)
        if cs == "lv03": return transform_xarray_coords_lv03_to_wgs84(da)
        if cs == "lv95": return transform_xarray_coords_lv95_to_wgs84(da)
        return None

    def load_lstm_ds(glacier, year):
        zpath = os.path.join(path_pred_lstm, glacier,
                             f"{glacier}_{year}_{period}.zarr")
        if not os.path.exists(zpath): return None
        ds = xr.open_zarr(zpath)
        if apply_smoothing_fn is not None:
            ds = apply_smoothing_fn(ds)
        return ds

    def lonlat_names(obj):
        coords = getattr(obj, "coords", {})
        if "lon" in coords and "lat" in coords: return "lon", "lat"
        if "longitude" in coords and "latitude" in coords:
            return "longitude", "latitude"
        return "lon", "lat"

    def stake_overlay_rmse(ax, glacier, year, cmap, norm, da_glamos, ds_lstm,
                           which):
        """Overlay stakes & return RMSE for 'GLAMOS' or 'LSTM' vs POINT_BALANCE."""
        if df_stakes is None:
            return None
        sub = df_stakes[(df_stakes.GLACIER == glacier)
                        & (df_stakes.YEAR == year)].copy()
        if period == "annual" and "PERIOD" in sub.columns:
            sub = sub[sub.PERIOD == "annual"].copy()
        if sub.empty:
            return None

        lx, ly = lonlat_names(
            ds_lstm if which == "LSTM" and ds_lstm is not None else da_glamos)

        def _safe_pred(ds, row):
            try:
                return get_predicted_mb(lx, ly, row, ds)
            except Exception:
                return np.nan

        def _safe_glamos(row):
            try:
                return get_predicted_mb_glamos(lx, ly, row, da_glamos)
            except Exception:
                return np.nan

        if which == "GLAMOS":
            sub["FIELD"] = sub.apply(_safe_glamos, axis=1)
        else:
            sub["FIELD"] = sub.apply(lambda r: _safe_pred(ds_lstm, r),
                                     axis=1) if ds_lstm is not None else np.nan

        hue_col = "POINT_BALANCE" if "POINT_BALANCE" in sub.columns else "FIELD"
        sns.scatterplot(
            data=sub,
            x="POINT_LON",
            y="POINT_LAT",
            hue=hue_col,
            palette=cmap,
            hue_norm=norm,
            ax=ax,
            s=18,
            legend=False,
        )

        if "POINT_BALANCE" in sub.columns and not np.all(np.isnan(
                sub["FIELD"])):
            return root_mean_squared_error(sub["POINT_BALANCE"], sub["FIELD"])
        return None

    # ---------- figure & gridspec (2 rows × 6 columns with CB slots) ----------
    fig = plt.figure(figsize=(28, 15))
    gs = GridSpec(
        nrows=2,
        ncols=6,
        figure=fig,
        width_ratios=[1, 1, 0.045, 1, 1, 0.045],  # slim cbar columns
        wspace=0.30,
        hspace=0.12)

    # Keep the first axis per row to share y across the row
    first_ax_in_row = [None, None]

    for r, glacier in enumerate(glacier_names):
        for j, year in enumerate(years):
            col_base = 3 * j  # (0 or 3)
            # sharey with the first axis in this row
            ax_g = fig.add_subplot(gs[r, col_base + 0],
                                   sharey=first_ax_in_row[r])
            if first_ax_in_row[r] is None:
                first_ax_in_row[
                    r] = ax_g  # set after creation of the first GLAMOS axis in row
            ax_m = fig.add_subplot(gs[r, col_base + 1],
                                   sharey=first_ax_in_row[r])
            ax_cb = fig.add_subplot(gs[r, col_base + 2])

            # load data
            da_g = load_glamos_wgs84(glacier, year)
            ds_m = load_lstm_ds(glacier, year)

            # compute pair vmin/vmax
            vals = []
            if da_g is not None:
                vals += [float(da_g.min().item()), float(da_g.max().item())]
            if ds_m is not None and "pred_masked" in ds_m:
                vals += [
                    float(ds_m["pred_masked"].min().item()),
                    float(ds_m["pred_masked"].max().item())
                ]
            if not vals:
                for ax in (ax_g, ax_m):
                    ax.text(0.5,
                            0.5,
                            f"No data\n{glacier} {year}",
                            ha="center",
                            va="center")
                    ax.set_axis_off()
                ax_cb.set_axis_off()
                continue
            vmin, vmax = min(vals), max(vals)
            cmap, norm, _, _ = get_color_maps(vmin, vmax, 0, 0)

            # --- GLAMOS panel ---
            mappable_g = None
            if da_g is None:
                ax_g.text(0.5,
                          0.5,
                          f"No GLAMOS\n{glacier} {year}",
                          ha="center",
                          va="center")
                ax_g.set_axis_off()
            else:
                mappable_g = da_g.plot.imshow(ax=ax_g,
                                              cmap=cmap,
                                              norm=norm,
                                              add_colorbar=False)
                ax_g.set_title(f"{glacier.capitalize()} – GLAMOS ({year})",
                               fontsize=16)

                mean_g = float(da_g.mean().item())
                var_g = float(da_g.var().item())
                rmse_g = stake_overlay_rmse(ax_g,
                                            glacier,
                                            year,
                                            cmap,
                                            norm,
                                            da_g,
                                            ds_m,
                                            which="GLAMOS")
                text_g = (f"RMSE: {rmse_g:.2f}\n" if rmse_g is not None else "") \
                         + f"mean MB: {mean_g:.2f}\nvar: {var_g:.2f}"
                ax_g.text(0.03,
                          0.03,
                          text_g,
                          transform=ax_g.transAxes,
                          ha="left",
                          va="bottom",
                          fontsize=12,
                          bbox=dict(facecolor="white",
                                    alpha=0.7,
                                    edgecolor="none"))

            # --- LSTM panel ---
            mappable_m = None
            if ds_m is None or "pred_masked" not in ds_m:
                ax_m.text(0.5,
                          0.5,
                          f"No LSTM\n{glacier} {year}",
                          ha="center",
                          va="center")
                ax_m.set_axis_off()
            else:
                mappable_m = ds_m["pred_masked"].plot.imshow(
                    ax=ax_m, cmap=cmap, norm=norm, add_colorbar=False)
                ax_m.set_title(f"{glacier.capitalize()} – LSTM ({year})",
                               fontsize=16)

                mean_m = float(ds_m["pred_masked"].mean().item())
                var_m = float(ds_m["pred_masked"].var().item())
                rmse_m = stake_overlay_rmse(ax_m,
                                            glacier,
                                            year,
                                            cmap,
                                            norm,
                                            da_g,
                                            ds_m,
                                            which="LSTM")
                text_m = (f"RMSE: {rmse_m:.2f}\n" if rmse_m is not None else "") \
                         + f"mean MB: {mean_m:.2f}\nvar: {var_m:.2f}"
                ax_m.text(0.03,
                          0.03,
                          text_m,
                          transform=ax_m.transAxes,
                          ha="left",
                          va="bottom",
                          fontsize=12,
                          bbox=dict(facecolor="white",
                                    alpha=0.7,
                                    edgecolor="none"))

            # --- shared colorbar outside (for this pair) ---
            pair_mappable = mappable_m or mappable_g
            if pair_mappable is not None:
                cb = fig.colorbar(pair_mappable, cax=ax_cb)
                cb.set_label("Mass Balance [m w.e.]", fontsize=16)

            # ---- tidy y labels: only leftmost panel shows them ----
            if j == 0:
                ax_g.set_ylabel("Latitude")
                ax_m.tick_params(labelleft=False)
                ax_m.set_ylabel("")
            else:
                # hide y labels/ticks to save space
                ax_g.tick_params(labelleft=False)
                ax_m.tick_params(labelleft=False)
                ax_g.set_ylabel("")
                ax_m.set_ylabel("")
            # always label x on bottom row only (optional; comment out if not desired)
            if r == 0:
                ax_g.tick_params(labelbottom=False)
                ax_m.tick_params(labelbottom=False)
            else:
                ax_g.set_xlabel("Longitude")
                ax_m.set_xlabel("Longitude")

    plt.tight_layout()
    plt.show()

In [ ]:
plot_2glaciers_2years_glamos_vs_lstm(
    glacier_names=("aletsch", "rhone"),
    years=(2016, 2022),
    cfg=cfg,
    df_stakes=df_stakes,  # or None
    path_distributed_mb=path_distributed_MB_glamos,
    path_pred_lstm=PATH_PREDICTIONS_LSTM_two_heads,
    period="annual",
    apply_smoothing_fn=apply_gaussian_filter  # or None
)

## Geodetic MB:

In [ ]:
# Compute RMSE and Pearson correlation
rmse_nn = root_mean_squared_error(df_lstm_two_heads["Geodetic MB"],
                                  df_lstm_two_heads["MBM MB"])
corr_nn = np.corrcoef(df_lstm_two_heads["Geodetic MB"],
                      df_lstm_two_heads["MBM MB"])[0, 1]

plot_mbm_vs_geodetic_by_area_bin(df_lstm_two_heads,
                                 bins=[0, 1, 5, 10, 100, np.inf],
                                 labels=['<1', '1-5', '5–10', '>10', '>100'],
                                 max_bins=4)